Author:Mohammadreza Tavasoli Naeini Advisor:Hanrui Wang

This tutorial shows; how to use the TrochQuantum library in building a quantum transformer.
This notebook is adapted, excerpted from https://github.com/rdisipio/qtransformer, written by Riccardo Di Sipio. 
In this notebook, we use the TorchQuantum library instead of the Pennylane library to build a quantum machine learning model.



## Introduction to Transformer

Nowadays, Transformer architecture is dominant in the field of sequential data processing, such as natural language processing. Transformers designed to effieciently analyze sequential data on GPUs. Therfore, we see stonishing results in translation, generation from the models build on transformer blocks.[1],[2]

Transformer block does not have memory to sequentially anaylize the data; however, it has position-dependent embedding, which is a kind of word embbeding that embed position of the input in the vector too. Furthermore, trnasformer block use self attation mechanisem to understand which part of the output is related to the input data. The core of transformer block is multi head attetion layer. In multi head atteation layer, three diffrent linear transformer, WQ,WK,and WV, applies to each word embbeding to tranform them to new  internal representation called Query (Q), Key (K), and Value (V ). These states are then passed to the function that calculates the attention weights.[1],[2]



<figure>
<img src="transformer.png" style="width:30%">
<figcaption align = "center"> transformer architecture </figcaption>
</figure>

## Quantum Transformer

We convert the classical transformer to the quantum enhanced transformer by replacing the linear transformations WQ, WK, and WV with Varitinal Quantum Circuits. [1] 

## Build and train a Quantum Transformer.

## Installation

Install torchquantum

In [26]:
!git clone https://github.com/mit-han-lab/torchquantum.git

Cloning into 'torchquantum'...
remote: Enumerating objects: 11649, done.
remote: Counting objects: 100% (539/539), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 11649 (delta 315), reused 456 (delta 255), pack-reused 11110
Receiving objects: 100% (11649/11649), 6.43 MiB | 3.37 MiB/s, done.
Resolving deltas: 100% (6472/6472), done.


In [27]:
%cd torchquantum

/Users/mohammad/.Trash/torchquantum 10-11-35-421/torchquantum


In [28]:
!pip install --editable .

Obtaining file:///Users/mohammad/.Trash/torchquantum%2010-11-35-421/torchquantum
  Attempting uninstall: torchquantum
    Found existing installation: torchquantum 0.1.2
    Uninstalling torchquantum-0.1.2:
      Successfully uninstalled torchquantum-0.1.2
  Running setup.py develop for torchquantum


## Setup
Our code requires torchquantum lib, and pytorch. We need torch, function from torch.nn.functional, optimizers(optim), torchquantum module, and torchtext.

In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torchquantum as tq
import torchquantum.functional as tqf
import argparse
import tqdm
import time

import torch
import torch.nn.functional as F
import torchtext.legacy
from torchtext.legacy import data, datasets, vocab

## Build MultiHead Attention

In [2]:
class MultiHeadAttentionBase(nn.Module):
    def __init__(self,
                 embed_dim: int,
                 num_heads: int,
                 dropout: float = 0.1,
                 mask=None,
                 use_bias=False):
        super(MultiHeadAttentionBase, self).__init__()

        assert embed_dim % num_heads == 0, f"Embedding dimension ({embed_dim}) should be divisible by number of heads ({num_heads})"

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads  # projection dimensions
        self.k_linear = None
        self.q_linear = None
        self.v_linear = None
        self.combine_heads = None
        self.dropout = nn.Dropout(dropout)
        self.attn_weights = None
    
    def separate_heads(self, x):
        '''
        split into N heads
        from (batch_size, seq_len, embed_dim)
        to   (batch_size, seq_len, num_heads, embed_dim)
        then transpose (1,2) to (batch_size, num_heads, seq_len, embed_dim)
        to make mat mult straightforward for each head
        '''
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.num_heads, self.d_k)
        return x.transpose(1, 2)

    def attention(self, query, key, value, mask=None, dropout=None):
        '''
        Attention(Q, K, V) = softmax(Q K^T / sqrt(d_k))V
        '''
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.d_k)
        # see also: https://tensorchiefs.github.io/dlday2018/tutorial/einsum.html
        #scores = torch.einsum('bijh, bkjh -> bikh', query, key) / math.sqrt(self.d_k)
        if mask is not None:
            mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)
        scores = F.softmax(scores, dim=-1)
        if dropout is not None:
            scores = dropout(scores)
        attn = torch.matmul(scores, value)
        return attn, scores
    
    def downstream(self, query, key, value, batch_size, mask=None):
        Q = self.separate_heads(query)
        K = self.separate_heads(key)
        V = self.separate_heads(value)

        x, self.attn_weights = self.attention(Q, K, V, mask, dropout=self.dropout)

        concat = x.transpose(1, 2).contiguous().view(batch_size, -1, self.embed_dim)

        return concat
        # output = self.combine_heads(concat)
        # return output

   # def forward(self, x, mask=None):
    #    raise NotImplementedError("Base class does not execute forward function.")
        
        

In [3]:
class MultiHeadAttentionClassical(MultiHeadAttentionBase):
    
    def __init__(self, embed_dim: int,
                 num_heads: int,
                 dropout=0.1,
                 mask=None,
                 use_bias=False):
        super(MultiHeadAttentionClassical, self).__init__(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout, mask=mask, use_bias=use_bias)

        self.k_linear = nn.Linear(embed_dim, embed_dim, bias=use_bias)
        self.q_linear = nn.Linear(embed_dim, embed_dim, bias=use_bias)
        self.v_linear = nn.Linear(embed_dim, embed_dim, bias=use_bias)
        self.combine_heads = nn.Linear(embed_dim, embed_dim, bias=use_bias)
    
    def forward(self, x, mask=None):
        batch_size, seq_len, embed_dim = x.size()
        assert embed_dim == self.embed_dim, f"Input embedding ({embed_dim}) does not match layer embedding size ({self.embed_dim})"

        K = self.k_linear(x)
        Q = self.q_linear(x)
        V = self.v_linear(x)

        x = self.downstream(Q, K, V, batch_size, mask)
        output = self.combine_heads(x)
        return output

Typically, a Quantum Neural Network module consists of three parts: encoder, ansatz, and measurement. We can create an encoder by passing a list of gates to tq.GeneralEncoder. Each entry in the list contains input_idx, func, and wires. Here, each qubit has a rotation-X gate. 8 RX gates in total. They can encode the 8 input data to the quantum state. Then we choose ansatz such that they are entangled between each other, rotated by an arbitrary angle. Finally, we perform Pauli-Z measurements on each qubit by creating a tq.MeasureAll module and passing tq.PauliZ to it. The measure function will return four expectation values from four qubits.


In [4]:
class MultiHeadAttentionQuantum(MultiHeadAttentionBase):
    class QLayer(tq.QuantumModule):
        def __init__(self):
            super().__init__()    
            self.n_wires = 8
            self.encoder = tq.GeneralEncoder(
        [   {'input_idx': [0], 'func': 'rx', 'wires': [0]},
            {'input_idx': [1], 'func': 'rx', 'wires': [1]},
            {'input_idx': [2], 'func': 'rx', 'wires': [2]},
            {'input_idx': [3], 'func': 'rx', 'wires': [2]},
            {'input_idx': [4], 'func': 'rx', 'wires': [4]},
            {'input_idx': [5], 'func': 'rx', 'wires': [5]},
             {'input_idx': [6], 'func': 'rx', 'wires': [6]},
          {'input_idx': [7], 'func': 'rx', 'wires': [7]},
        ])
            self.rx0 = tq.RX(has_params=True, trainable=True)
            self.rx1 = tq.RX(has_params=True, trainable=True)
            self.rx2 = tq.RX(has_params=True, trainable=True)
            self.rx3 = tq.RX(has_params=True, trainable=True)
            self.rx4 = tq.RX(has_params=True, trainable=True)
            self.rx5 = tq.RX(has_params=True, trainable=True)
            self.rx6 = tq.RX(has_params=True, trainable=True)
            self.rx7 = tq.RX(has_params=True, trainable=True)
            self.measure = tq.MeasureAll(tq.PauliZ)
        def forward (self, x, q_device: tq.QuantumDevice):
            self.q_device = q_device
            self.encoder(self.q_device, x)
            self.rx0(self.q_device, wires=0)
            self.rx1(self.q_device, wires=1)
            self.rx2(self.q_device, wires=2)
            self.rx3(self.q_device, wires=3)
            self.rx4(self.q_device, wires=4)
            self.rx5(self.q_device, wires=5)
            self.rx6(self.q_device, wires=6)
            self.rx7(self.q_device, wires=7)
            for k in range(self.n_wires):
                if k==self.n_wires-1:
                    tqf.cnot(self.q_device, wires=[k, 0]) 
                else:
                    tqf.cnot(self.q_device, wires=[k, k+1])
            return(self.measure(self.q_device))
            
    def __init__(self,
                 embed_dim: int,
                 num_heads: int,
                 dropout=0.1,
                 mask=None,
                 use_bias=False,
                 n_qubits: int = 4,
                 n_qlayers: int = 1,
                 q_device="default.qubit"):
        super(MultiHeadAttentionQuantum, self).__init__(embed_dim, num_heads, dropout=dropout, mask=mask, use_bias=use_bias)
        
        # todo: add intermediate layer to "dress" quantum circuit
        assert n_qubits == embed_dim, "Number of qubits ({n_qubits}) does not match embedding dim ({embed_dim})"
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.dev = tq.QuantumDevice(self.n_qubits)
        self.q_layer = self.QLayer()
        self.measure = tq.MeasureAll(tq.PauliZ)
    def forward(self, x, mask=None):
        batch_size, seq_len, embed_dim = x.size()
        assert embed_dim == self.embed_dim, f"Input embedding ({embed_dim}) does not match layer embedding size ({self.embed_dim})"

        K = [self.q_layer(x[:, t, :].clone(),self.dev) for t in range(seq_len)]
        Q = [self.q_layer(x[:, t, :].clone(),self.dev) for t in range(seq_len)]
        V = [self.q_layer(x[:, t, :].clone(),self.dev) for t in range(seq_len)]
        K = torch.Tensor(pad_sequence(K))
        Q = torch.Tensor(pad_sequence(Q))
        V = torch.Tensor(pad_sequence(V))
        x = self.downstream(Q, K, V, batch_size, mask)
        output = [self.q_layer(x[:, t, :],self.dev) for t in range(seq_len)]
        output = torch.Tensor(pad_sequence(output)).clone()
        return output

## Build FeedForwrad Layer

In [5]:
class FeedForwardBase(nn.Module):
    def __init__(self, embed_dim, ffn_dim, dropout=0.1):
        super(FeedForwardBase, self).__init__()
        self.linear_1 = nn.Linear(embed_dim, ffn_dim)
        self.linear_2 = nn.Linear(ffn_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    #def forward(self, x):
     #   raise NotImplementedError("Base class does not implement forward function")


class FeedForwardClassical(FeedForwardBase):
    def __init__(self, embed_dim, ffn_dim, dropout=0.1):
        super(FeedForwardClassical, self).__init__(embed_dim, ffn_dim, dropout)

    def forward(self, x):
        x = F.relu(self.linear_1(x))
        x = self.dropout(x)
        x = self.linear_2(x)
        return x

Here, similar to what we have done for the multi-head attention layer, we build the FeedForward quantum layer.




In [6]:
class FeedForwardQuantum(FeedForwardBase):
    class QL(tq.QuantumModule):
        def __init__(self):
            super().__init__()    
            self.n_wires = 8
            self.encoder = tq.GeneralEncoder(
        [   {'input_idx': [0], 'func': 'rx', 'wires': [0]},
            {'input_idx': [1], 'func': 'rx', 'wires': [1]},
            {'input_idx': [2], 'func': 'rx', 'wires': [2]},
            {'input_idx': [3], 'func': 'rx', 'wires': [2]},
            {'input_idx': [4], 'func': 'rx', 'wires': [4]},
            {'input_idx': [5], 'func': 'rx', 'wires': [5]},
             {'input_idx': [6], 'func': 'rx', 'wires': [6]},
          {'input_idx': [7], 'func': 'rx', 'wires': [7]},
        ])
            self.rx0 = tq.RX(has_params=True, trainable=True)
            self.rx1 = tq.RX(has_params=True, trainable=True)
            self.rx2 = tq.RX(has_params=True, trainable=True)
            self.rx3 = tq.RX(has_params=True, trainable=True)
            self.rx4 = tq.RX(has_params=True, trainable=True)
            self.rx5 = tq.RX(has_params=True, trainable=True)
            self.rx6 = tq.RX(has_params=True, trainable=True)
            self.rx7 = tq.RX(has_params=True, trainable=True)
            self.measure = tq.MeasureAll(tq.PauliZ)
        def forward (self, x, q_device: tq.QuantumDevice):
            self.q_device = q_device
            self.encoder(self.q_device, x)
            self.rx0(self.q_device, wires=0)
            self.rx1(self.q_device, wires=1)
            self.rx2(self.q_device, wires=2)
            self.rx3(self.q_device, wires=3)
            self.rx4(self.q_device, wires=4)
            self.rx5(self.q_device, wires=5)
            self.rx6(self.q_device, wires=6)
            self.rx7(self.q_device, wires=7)
            for k in range(self.n_wires):
                if k==self.n_wires-1:
                    tqf.cnot(self.q_device, wires=[k, 0]) 
                else:
                    tqf.cnot(self.q_device, wires=[k, k+1])
            return(self.measure(self.q_device))
                    
    def __init__(self, embed_dim, n_qubits, n_qlayers=1, dropout=0.1, q_device="default.qubit"):
        
        super(FeedForwardQuantum, self).__init__(embed_dim, ffn_dim=n_qubits, dropout=dropout)
        
        assert n_qubits == embed_dim, "Number of qubits ({n_qubits}) does not match embedding dim ({embed_dim})"
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.dev = tq.QuantumDevice(self.n_qubits)
        self.q_l = self.QL()
    
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        x = self.linear_1(x)
        o = [self.q_l(x[:, t, :].detach(),self.dev) for t in range(seq_len)]
        x = torch.Tensor(pad_sequence(o))
        x = self.linear_2(x)
        return x

## Build the whole hybrid transformer model.

In [7]:
class TransformerBlockBase(nn.Module):
    def __init__(self,
                 embed_dim: int,
                 num_head: int,
                 ff_dim: int,
                 n_qubits_transformer: int = 0,
                 n_qubits_ffn: int = 0,
                 n_qlayers: int = 1,
                 dropout: float = 0.1,
                 mask=None):
        super(TransformerBlockBase, self).__init__()
        self.attn = None
        self.ffn = None
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        attn_output = self.attn(x)
        x = self.norm1(attn_output + x)
        x = self.dropout1(x)

        ff_output = self.ffn(x)
        x = self.norm2(ff_output + x)
        x = self.dropout2(x)

        return x
class TransformerBlockClassical(TransformerBlockBase):
    def __init__(self,
                 embed_dim: int,
                 num_heads: int,
                 ff_dim: int,
                 dropout: float = 0.1,
                 mask=None):
        super(TransformerBlockClassical, self).__init__(embed_dim, num_heads, ff_dim, dropout, mask)
        self.attn = MultiHeadAttentionClassical(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout, mask=mask)
        self.ffn = FeedForwardClassical(embed_dim, ff_dim)

class TransformerBlockQuantum(TransformerBlockBase):
    def __init__(self,
                 embed_dim: int,
                 num_heads: int,
                 ffn_dim: int,
                 n_qubits_transformer: int = 0,
                 n_qubits_ffn: int = 0,
                 n_qlayers: int = 1,
                 dropout: float = 0.1,
                 mask=None,
                 q_device='default.qubit'):
        super(TransformerBlockQuantum, self).__init__(embed_dim, num_heads, ffn_dim, dropout, mask)
        
        self.n_qubits_transformer = n_qubits_transformer
        self.n_qubits_ffn = n_qubits_ffn
        self.n_qlayers = n_qlayers

        self.attn = MultiHeadAttentionQuantum(embed_dim,
                                              num_heads,
                                              n_qubits=n_qubits_transformer,
                                              n_qlayers=n_qlayers,
                                              dropout=dropout,
                                              mask=mask,
                                              q_device=q_device)
        if n_qubits_ffn > 0:
            self.ffn = FeedForwardQuantum(embed_dim, n_qubits_ffn, n_qlayers, q_device=q_device)
        else:
            self.ffn = FeedForwardClassical(embed_dim, ffn_dim)


class PositionalEncoder(nn.Module):
    def __init__(self, embed_dim, max_seq_len=512):
        super().__init__()
        self.embed_dim = embed_dim
        
        # create constant 'pe' matrix with values dependant on pos and i
        pe = torch.zeros(max_seq_len, embed_dim)
        for pos in range(max_seq_len):
            for i in range(0, embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/embed_dim)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.embed_dim)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)  # .cuda()
        return x


class TextClassifier(nn.Module):
    def __init__(self,
                 embed_dim: int,
                 num_heads: int,
                 num_blocks: int,
                 num_classes: int,
                 vocab_size: int,
                 ffn_dim: int = 32,
                 n_qubits_transformer: int = 0,
                 n_qubits_ffn: int = 0,
                 n_qlayers: int = 1,
                 dropout=0.1,
                 q_device="device.qubit"):
        super(TextClassifier, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.num_classes = num_classes
        self.vocab_size = vocab_size

        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = PositionalEncoder(embed_dim)

        print(f"++ There will be {num_blocks} transformer blocks")

        if n_qubits_transformer > 0:
            print(f"++ Transformer will use {n_qubits_transformer} qubits and {n_qlayers} q layers")
            if n_qubits_ffn > 0:
                print(f"The feed-forward head will use {n_qubits_ffn} qubits")
            else:
                print(f"The feed-forward head will be classical")
            print(f"Using quantum device {q_device}")

            transformer_blocks = [
                TransformerBlockQuantum(embed_dim, num_heads, ffn_dim,
                                        n_qubits_transformer=n_qubits_transformer,
                                        n_qubits_ffn=n_qubits_ffn,
                                        n_qlayers=n_qlayers, 
                                        q_device=q_device) for _ in range(num_blocks)
            ]
        else:
            transformer_blocks = [
                TransformerBlockClassical(embed_dim, num_heads, ffn_dim) for _ in range(num_blocks)
            ]

        self.transformers = nn.Sequential(*transformer_blocks)
        if self.num_classes > 2:
            self.class_logits = nn.Linear(embed_dim, num_classes)
        else:
            self.class_logits = nn.Linear(embed_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        tokens = self.token_embedding(x)
        # batch_size, seq_len, embed_dim = x.size()
        x = self.pos_embedding(tokens)
        x = self.transformers(x)
        x = x.mean(dim=1)  # global average pooling, works in 1D
        x = self.dropout(x)
        # x = self.class_logits(x)
        # return F.log_softmax(x, dim=1)
        return self.class_logits(x)
        

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



In [9]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc




In [10]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for batch in iterator:
        torch.autograd.set_detect_anomaly(True)
        optimizer.zero_grad()

        inputs = torch.LongTensor(batch.text[0])
        if inputs.size(1) > MAX_SEQ_LEN:
            inputs = inputs[:, :MAX_SEQ_LEN]
        predictions = model(inputs).squeeze(1)
        
        label = batch.label - 1
        #label = label.unsqueeze(1)
        loss = criterion(predictions, label)
        #loss = F.nll_loss(predictions, label)
        acc = binary_accuracy(predictions, label)
        
        loss.backward(retain_graph=True)
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [11]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            inputs = torch.LongTensor(batch.text[0])
            if inputs.size(1) > MAX_SEQ_LEN:
                inputs = inputs[:, :MAX_SEQ_LEN]
            predictions = model(inputs).squeeze(1)
            
            label = batch.label - 1
            #label = label.unsqueeze(1)
            loss = criterion(predictions, label)
            #loss = F.nll_loss(predictions, label)
            acc = binary_accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)




In [12]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs




## Training Classical transformer

In [22]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser() 
    parser.add_argument('-D', '--q_device', default='local', type=str)
    parser.add_argument('-B', '--batch_size', default=32, type=int)
    parser.add_argument('-E', '--n_epochs', default=5, type=int)
    parser.add_argument('-C', '--n_classes', default=2, type=int)
    parser.add_argument('-l', '--lr', default=0.001, type=float)
    parser.add_argument('-v', '--vocab_size', default=20000, type=int)
    parser.add_argument('-e', '--embed_dim', default=8, type=int)
    parser.add_argument('-s', '--max_seq_len', default=64, type=int)
    parser.add_argument('-f', '--ffn_dim', default=8, type=int)
    parser.add_argument('-t', '--n_transformer_blocks', default=1, type=int)
    parser.add_argument('-H', '--n_heads', default=2, type=int)
    parser.add_argument('-q', '--n_qubits_transformer', default=0, type=int)
    parser.add_argument('-Q', '--n_qubits_ffn', default=0, type=int)
    parser.add_argument('-L', '--n_qlayers', default=1, type=int)
    parser.add_argument('-d', '--dropout_rate', default=0.1, type=float)
    args = parser.parse_args(args=[])

    MAX_SEQ_LEN = args.max_seq_len

    TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
    #LABEL = data.Field(sequential=False)
    LABEL = data.LabelField(dtype=torch.float)
    train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    print(f'Training examples: {len(train_data)}')
    print(f'Testing examples:  {len(test_data)}')

    TEXT.build_vocab(train_data, max_size=args.vocab_size - 2)  # exclude <UNK> and <PAD>
    LABEL.build_vocab(train_data)

    train_iter, test_iter = data.BucketIterator.splits((train_data, test_data), batch_size=args.batch_size)
    
    model = TextClassifier(embed_dim=args.embed_dim,
                           num_heads=args.n_heads,
                           num_blocks=args.n_transformer_blocks,
                           num_classes=args.n_classes,
                           vocab_size=args.vocab_size,
                           ffn_dim=args.ffn_dim,
                           n_qubits_transformer=args.n_qubits_transformer,
                           n_qubits_ffn=args.n_qubits_ffn,
                           n_qlayers=args.n_qlayers,
                           dropout=args.dropout_rate,
                           q_device=args.q_device)
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = torch.optim.Adam(lr=args.lr, params=model.parameters())
    if args.n_classes < 3:
        criterion = torch.nn.BCEWithLogitsLoss()  # logits -> sigmoid -> loss
    else:
        criterion = torch.nn.CrossEntropyLoss()  # logits -> log_softmax -> NLLloss

    # training loop
    best_valid_loss = float('inf')
    for iepoch in range(args.n_epochs):
        start_time = time.time()

        print(f"Epoch {iepoch+1}/{args.n_epochs}")

        train_loss, train_acc = train(model, train_iter, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, test_iter, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'model.pt')
        
        print(f'Epoch: {iepoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')




Training examples: 25000
Testing examples:  25000
++ There will be 1 transformer blocks
The model has 160,441 trainable parameters
Epoch 1/5
Epoch: 01 | Epoch Time: 0m 28s
	Train Loss: -5.346 | Train Acc: 49.22%
	 Val. Loss: -13.767 |  Val. Acc: 50.04%
Epoch 2/5
Epoch: 02 | Epoch Time: 0m 31s
	Train Loss: -28.989 | Train Acc: 50.00%
	 Val. Loss: -47.243 |  Val. Acc: 50.04%
Epoch 3/5
Epoch: 03 | Epoch Time: 0m 28s
	Train Loss: -70.884 | Train Acc: 50.01%
	 Val. Loss: -97.031 |  Val. Acc: 50.04%
Epoch 4/5
Epoch: 04 | Epoch Time: 0m 28s
	Train Loss: -127.913 | Train Acc: 50.01%
	 Val. Loss: -160.671 |  Val. Acc: 50.04%
Epoch 5/5
Epoch: 05 | Epoch Time: 0m 28s
	Train Loss: -197.544 | Train Acc: 50.01%
	 Val. Loss: -236.139 |  Val. Acc: 50.04%


## Training Quantum transformer

In [ ]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser() 
    parser.add_argument('-D', '--q_device', default='default.qubit', type=str)
    parser.add_argument('-B', '--batch_size', default=32, type=int)
    parser.add_argument('-E', '--n_epochs', default=1, type=int)
    parser.add_argument('-C', '--n_classes', default=2, type=int)
    parser.add_argument('-l', '--lr', default=0.001, type=float)
    parser.add_argument('-v', '--vocab_size', default=50000, type=int)
    parser.add_argument('-e', '--embed_dim', default=8, type=int)
    parser.add_argument('-s', '--max_seq_len', default=64, type=int)
    parser.add_argument('-f', '--ffn_dim', default=8, type=int)
    parser.add_argument('-t', '--n_transformer_blocks', default=1, type=int)
    parser.add_argument('-H', '--n_heads', default=2, type=int)
    parser.add_argument('-q', '--n_qubits_transformer', default=8, type=int)
    parser.add_argument('-Q', '--n_qubits_ffn', default=8, type=int)
    parser.add_argument('-L', '--n_qlayers', default=1, type=int)
    parser.add_argument('-d', '--dropout_rate', default=0.1, type=float)
    args = parser.parse_args(args=[])

    MAX_SEQ_LEN = args.max_seq_len

    TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
    #LABEL = data.Field(sequential=False)
    LABEL = data.LabelField(dtype=torch.float)
    train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    print(f'Training examples: {len(train_data)}')
    print(f'Testing examples:  {len(test_data)}')

    TEXT.build_vocab(train_data, max_size=args.vocab_size - 2)  # exclude <UNK> and <PAD>
    LABEL.build_vocab(train_data)

    train_iter, test_iter = data.BucketIterator.splits((train_data, test_data), batch_size=args.batch_size)
    
    model = TextClassifier(embed_dim=args.embed_dim,
                           num_heads=args.n_heads,
                           num_blocks=args.n_transformer_blocks,
                           num_classes=args.n_classes,
                           vocab_size=args.vocab_size,
                           ffn_dim=args.ffn_dim,
                           n_qubits_transformer=args.n_qubits_transformer,
                           n_qubits_ffn=args.n_qubits_ffn,
                           n_qlayers=args.n_qlayers,
                           dropout=args.dropout_rate,
                           q_device=args.q_device)
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = torch.optim.Adam(lr=args.lr, params=model.parameters())
    if args.n_classes < 3:
        criterion = torch.nn.BCEWithLogitsLoss()  # logits -> sigmoid -> loss
    else:
        criterion = torch.nn.CrossEntropyLoss()  # logits -> log_softmax -> NLLloss

    # training loop
    best_valid_loss = float('inf')
    for iepoch in range(args.n_epochs):
        start_time = time.time()

        print(f"Epoch {iepoch+1}/{args.n_epochs}")

        train_loss, train_acc = train(model, train_iter, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, test_iter, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'model.pt')
        
        print(f'Epoch: {iepoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')





Training examples: 25000
Testing examples:  25000
++ There will be 1 transformer blocks
++ Transformer will use 8 qubits and 1 q layers
The feed-forward head will use 8 qubits
Using quantum device default.qubit
The model has 400,201 trainable parameters
Epoch 1/1


## Refrences

[1] Riccardo Di Sipio, Jia-Hong Huang, Samuel Yen-
Chi Chen, Stefano Mangini, and Marcel Worring.
The dawn of quantum natural language processing.
In ICASSP 2022-2022 IEEE International Confer-
ence on Acoustics, Speech and Signal Processing
(ICASSP), pages 8612–8616. IEEE, 2022.

[2] Ashish Vaswani, Noam Shazeer, Niki Parmar,
Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
Łukasz Kaiser, and Illia Polosukhin. Attention is all
you need. In Advances in neural information processing systems, pages 5998–6008, 2017.